In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

train = pd.read_table("train.tsv")
test = pd.read_table("test.tsv")

#特徴量と目的変数を分ける。(x:特徴量、y:目的変数)

train_x = train.copy()
train_y = train['class']
test_x = test.copy().drop(['id'], axis=1)
#train_x['class'].unique()
#train_x['buying'].unique() 
#train_x['maint'].unique()
#train_x['doors'].unique()
#train_x['persons'].unique()
#train_x['lug_boot'].unique()
#train_x['safety'].unique()

#辞書づくり
class_dic = {'unacc':0,'acc':1,'good':2,'vgood':3}
buying_dic = {'low':0,'med':1,'high':2,'vhigh':3}
maint_dic = {'low':0,'med':1,'high':2,'vhigh':3}
lug_boot_dic = {'small':0,'med':1,'big':2}
doors_dic = {'3':3, '5more':5, '2':2, '4':4}
persons_dic = {'2':2, 'more':6, '4':4}
safety_dic = {'low':0,'med':1,'high':2}

#数値化
train_x['class'] = train_x['class'].map(class_dic)
train_x['buying'] = train_x['buying'].map(buying_dic)
train_x['maint'] = train_x['maint'].map(maint_dic)
train_x['lug_boot'] = train_x['lug_boot'].map(lug_boot_dic)
train_x['safety'] = train_x['safety'].map(safety_dic)
train_x['doors'] = train_x['doors'].map(doors_dic)
train_x['persons'] = train_x['persons'].map(persons_dic)

train_y = train_y.map(class_dic)

test_x['buying'] = test_x['buying'].map(buying_dic)
test_x['maint'] = test_x['maint'].map(maint_dic)
test_x['lug_boot'] = test_x['lug_boot'].map(lug_boot_dic)
test_x['safety'] = test_x['safety'].map(safety_dic)
test_x['doors'] = test_x['doors'].map(doors_dic)
test_x['persons'] = test_x['persons'].map(persons_dic)

# from sklearn.preprocessing import LabelEncoder
# train_x.loc[train_x['doors'] == '5more', 'doors'] = 5
# train_x.loc[train_x['persons'] == 'more', 'persons'] = 5
# train_x.loc[train_x['buying'] == 'low', 'buying'] = 1
# train_x.loc[train_x['buying'] == 'med', 'buying'] = 2
# train_x.loc[train_x['buying'] == 'high', 'buying'] = 3
# train_x.loc[train_x['buying'] == 'vhigh', 'buying'] = 4
# train_x.loc[train_x['maint'] == 'low', 'maint'] = 1
# train_x.loc[train_x['maint'] == 'med', 'maint'] = 2
# train_x.loc[train_x['maint'] == 'high', 'maint'] = 3
# train_x.loc[train_x['maint'] == 'small', 'lug_boot'] = 1
# train_x.loc[train_x['maint'] == 'med', 'lug_boot'] = 2
# train_x.loc[train_x['maint'] == 'big', 'lug_boot'] = 3
# safetyとclassも
train_x.head()


,id,class,buying,maint,doors,persons,lug_boot,safety
0,0,0,0,1,3,2,0,0
1,3,1,0,2,3,6,0,1
2,7,0,3,2,5,2,0,1
3,11,1,2,2,3,6,2,1
4,12,0,2,2,3,2,1,2


In [20]:
#学習データからidとclassを除く
train_x = train_x.drop(['id','class'], axis=1)
train_x.head(30)
train_x_hist, train_x_bins = np.histogram(train_x)

import matplotlib.pyplot as plt


In [21]:
#特徴量作成
#標準化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_x)

train_x_standard = scaler.transform(train_x)
test_x_standard = scaler.transform(test_x)

train_x_hist, train_x_bins = np.histogram(train_x_standard)

import matplotlib.pyplot as plt
#plt.hist(train_x_standard, bins=train_x_bins)

#ロジスティック回帰
from sklearn.linear_model import  LogisticRegression

model_lg = LogisticRegression(C=100)
model_lg.fit(train_x_standard,train_y)

pred = model_lg.predict_proba(test_x_standard)
pred_flag = model_lg.predict(test_x_standard)

arrays_max = np.max(pred, axis=1)
length = len(arrays_max)

#predの予測結果をフラグ化。
for i in range(0,length):
    for j in range(0,4):
        if pred[i][j] >= arrays_max[i]:
            pred[i][j] = 1
        elif pred[i][j] < arrays_max[i]:
            pred[i][j] = 0

to_df = []

for i in range(0,length):
    for j in range(0,4):
        if pred[i][j] == 1:
            to_df.append(j)
            
df_array = np.array(to_df)
print(df_array[0:70])
print(pred_flag[0:70])


[0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 2 0 3 1 0 0 0 1
 2 0 0 1 0 0 0 3 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 2 0 3 1 0 0 0 1
 2 0 0 1 0 0 0 3 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0]


In [22]:
#xgboostで学習
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators=20,random_state=71)
model_xgb.fit(train_x,train_y)

#表記
pred_xgb = model_xgb.predict_proba(test_x)[:, ]
pred_flag_xgb =  model_xgb.predict(test_x)[:,]
pred_xgb


array([[0.8537451 , 0.0495311 , 0.04832133, 0.04840248],
       [0.5316372 , 0.31888726, 0.07387395, 0.07560159],
       [0.8702947 , 0.0439173 , 0.04285801, 0.04292998],
       ...,
       [0.8697772 , 0.04448587, 0.04283252, 0.04290446],
       [0.8730448 , 0.04299501, 0.04194487, 0.04201531],
       [0.8494118 , 0.04344425, 0.04202991, 0.06511405]], dtype=float32)

In [23]:
feature_importance = model_xgb.feature_importances_
importance = pd.DataFrame({'features':test_x.columns,'importance':feature_importance})
importance


,features,importance
0,buying,0.119520
1,maint,0.106504
2,doors,0.000000
3,persons,0.354553
4,lug_boot,0.082167
5,safety,0.337257
